In [1]:
!pip install -U langchain-community
!pip install langchain_experimental
!pip install --upgrade "httpx<0.28"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [2]:
#@title #Load Libraries


import json

import os
import yaml

from langchain.agents import create_json_agent, AgentExecutor
from langchain.agents.agent_toolkits import JsonToolkit
from langchain.chains import LLMChain
from langchain.llms.openai import OpenAI
from langchain.requests import TextRequestsWrapper
from langchain.tools.json.tool import JsonSpec
from collections import defaultdict
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain_experimental.agents import create_csv_agent, create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain.agents import load_tools
import time

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load the Data (Takes about 5 minutes to upload)

In [3]:
from google.colab import files
uploaded = files.upload()

Saving parkinsons_data.csv to parkinsons_data.csv


In [ ]:
open_api_key = "" # cannot share open_api_key

In [5]:
orders_df = pd.read_csv('/content/parkinsons_data.csv')

In [6]:
# @title Load the AI Agent and begin asking questions

agent = create_csv_agent(
    ChatOpenAI(openai_api_key=open_api_key, temperature=0, model="gpt-3.5-turbo"),
    ['/content/parkinsons_data.csv'],
    verbose=False,
    allow_dangerous_code=True
    #agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
   # agent_type=AgentType.OPENAI_FUNCTIONS,
)


<ipython-input-6-6399ec5083ae>:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  ChatOpenAI(openai_api_key=open_api_key, temperature=0, model="gpt-3.5-turbo"),


In [ ]:
# @title Load the AI Agent and begin asking questions

open_api_key = "" # cannot share open_api_key


agent = create_csv_agent(
    ChatOpenAI(openai_api_key=open_api_key, temperature=0, model='gpt-4-1106-preview'),
    ['/content/parkinsons_data.csv'], allow_dangerous_code=True,
    verbose=False,
    #agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
   # agent_type=AgentType.OPENAI_FUNCTIONS,
)


In [ ]:
from openai import OpenAI
import os

client = OpenAI(
    api_key = "" # cannot share open_api_key
)

models = client.models.list()

print(models)

SyncPage[Model](data=[Model(id='gpt-4o-mini-audio-preview-2024-12-17', created=1734115920, object='model', owned_by='system'), Model(id='gpt-4o-2024-05-13', created=1715368132, object='model', owned_by='system'), Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'), Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='gpt-4o-audio-preview-2024-10-01', created=1727389042, object='model', owned_by='system'), Model(id='gpt-4o-audio-preview', created=1727460443, object='model', owned_by='system'), Model(id='gpt-4o', created=1715367049, object='model', owned_by='system'), Model(id='o1-mini-2024-09-12', created=1725648979, object='model', owned_by='system'), Model(id='gpt-4o-mini-realtime-preview-2024-12-17', created=1734112601, object='model', owned_by='system'), Model(id='o1-preview-2024-09-12', created=1725648865, object='model', owned_by='sys

In [10]:
query_str = 'Which features in the dataset correlate the most and which ones correlate the least?'  #@param

try:
    response = agent.invoke({"input": query_str})
except ValueError as e:
    response = str(e)
    if not response.startswith("Could not parse LLM output: `"):
        raise e
    response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")
print(response)

{'input': 'Which features in the dataset correlate the most and which ones correlate the least?', 'output': "The features that correlate the most in the dataset are 'Shimmer:APQ3' and 'Shimmer:DDA' with a correlation coefficient of approximately 1. The features that correlate the least are 'MDVP:Shimmer' and 'HNR' with a correlation coefficient of approximately -0.835."}
